# Example with Lakeshore Model 625 Superconducting Magnet power supply

In [1]:
import qcodes as qc

In [2]:
from qcodes_contrib_drivers.drivers.Lakeshore.Model_625 import Lakeshore625

In [3]:
# Initialize a single power supply for a superconducting magnet using GPIB
# specify coil constant of the magnet in units of T/A
# specifz field ramp rate in units of T/min
magnet = Lakeshore625(name = 'magnet', address = 'GPIB0::4::INSTR', coil_constant = 0.0166614, field_ramp_rate = 0.15)

c:\users\g-gre-gre054789\qcodes\qcodes\instrument\parameter.py:258: UserWarning: _BaseParameter got unexpected kwargs: {'full_name': 'magnet_field'}. These are unused and will be discarded. This will be an error in the future.
  warnings.warn(f"_BaseParameter got unexpected kwargs: {kwargs}."


Connected to: LSCI 625 (serial:6251287, firmware:1.3/1.1) in 0.73s


In [4]:
magnet.IDN()

{'vendor': 'LSCI', 'model': '625', 'serial': '6251287', 'firmware': '1.3/1.1'}

In [5]:
# Let's look at all parameters
magnet.print_readable_snapshot(update=True)

magnet:
	parameter                value
--------------------------------------------------------------------------------
IDN                       :	{'vendor': 'LSCI', 'model': '625', 'serial': '625...
coil_constant             :	0.01666 (magnet_coil_constant_unit)
coil_constant_unit        :	T/A 
current                   :	0.6019 (A)
current_limit             :	55 (A)
current_ramp_rate         :	0.15 (A/s)
current_rate_limit        :	0.3 (A/s)
field                     :	0.010028 (T)
field_ramp_rate           :	0.14994 (T/min)
oer_quench                :	no quench detected 
operational_error_status  :	000000000 
persistent_switch_heater  :	disabled 
quench_current_step_limit :	0.4 (A/s)
quench_detection          :	enabled 
ramp_segments             :	disabled 
ramping_state             :	not ramping 
timeout                   :	10 (s)
voltage                   :	0.1063 (V)
voltage_limit             :	1 (V)


## Check current to field conversion

In [6]:
# Since the set method of the driver only excepts fields in Tesla and we want to check if the correct 
# currents are applied, we need to convert target currents to target fields. For this reason we need 
# the coil constant. 
coil_const = magnet.coil_constant()
current_limit = magnet.current_limit()
current_rate_limit = magnet.current_rate_limit()
print("coil constant = {} T/A".format(coil_const))
print("current limit = {} A".format(current_limit))
print("current ramp rate limit = {} A/s".format(current_rate_limit))

coil constant = 0.01666 T/A
current limit = 55.0 A
current ramp rate limit = 0.3 A/s


In [7]:
# Let see if we can set and get the field in Tesla 
target_current = 0.1  # [A]  The current we want to set 
target_field = coil_const * target_current  # [T]
print("Target field is {} T".format(target_field))
magnet.field(target_field)

field = magnet.field()  # This gives us the measured field
print("Measured field is {} T".format(field))
# The current should be 
current = field / coil_const
print("Measured current is = {} A".format(current))
# We have verified with manual inspection that the current has indeed ben reached

Target field is 0.0016660000000000002 T
Measured field is 0.001685 T
Measured current is = 0.10114045618247298 A


## Let's have a look at other parameters

### Field

In [8]:
# Let's read the field
magnet.field()

0.001685

In [9]:
# Let's set a field (blocking mode)
magnet.field(0.005)

In [10]:
# Let's set a field (non-blocking mode)
magnet.set_field(0.01, block=False)

### Ramp rate

In [11]:
# The field ramp rate can easily be changed (this will update the current ramp rate as well since the power supply only works in current)
magnet.field_ramp_rate(0.1)
print('Field ramp rate: {} {}.'.format(magnet.field_ramp_rate(), magnet.field_ramp_rate.unit))
print('Current ramp rate: {} {}.'.format(magnet.current_ramp_rate(), magnet.current_ramp_rate.unit))

magnet.field_ramp_rate(0.2)
print('Field ramp rate: {} {}.'.format(magnet.field_ramp_rate(), magnet.field_ramp_rate.unit))
print('Current ramp rate: {} {}.'.format(magnet.current_ramp_rate(), magnet.current_ramp_rate.unit))

Field ramp rate: 0.09996000000000001 T/min.
Current ramp rate: 0.1 A/s.
Field ramp rate: 0.19992000000000001 T/min.
Current ramp rate: 0.2 A/s.


### Quench detection

In [12]:
# Quenches detection can be enabled, which it is by default
print('Quench detection is {}.'.format(magnet.quench_detection()))

# A quench is detected if the current rate is above the follwoing limit
print('Current rate limit for quench detection:{} {}'.format(magnet.quench_current_step_limit(), magnet.quench_current_step_limit.unit))

# Let's check if there was a quench detected
print('There was no {}.'.format(magnet.oer_quench()))

Quench detection is enabled.
Current rate limit for quench detection:0.4 A/s
There was no no quench detected.
